# Simulations naïves de files d'attentes

Ce TP vous invite à explorer les files d'attentes à travers des simulations (souvent naïves) de celles-ci. L'étude des systèmes d'attente ne peut se contenter d'un travail d'analyse théorique. Ces analyses sont souvent non disponibles et leur production nécessitent un savoir-faire mathématique expert. La simulation dans ce contexte permet d'apporter un premier regard sur les phénomènes à l'observation, même si elles n'apporte pas de réponses définitives elles sont souvent suffisantes pour faire un choix en première approximation. À minimna une simulation permet une meilleure qualification des variables en jeux lors d'une potentielle mise à contribution d'une équipe spécialisée. 

-----------------------

## Générer de l'aléatoire

Simuler une file d'attente sous-entend simuler un phénomène aléatoire. Pour pouvoir aborder sereinement cette simulation, on s'autorise à utiliser les générateurs aléatoires à disposition dans `numpy`. De toute manière, vous savez déjà construire un générateur aléatoire à partir d'une loi uniforme, *n'est-ce pas?*

In [2]:
import numpy as np
import pandas as pd

Pour référence, on revient rapidement sur la simulation d'une variable aléatoire suivant une loi disponibles dans `numpy`. Il s'agit en premier lieu d'initialiser un générateur aléatoire, afin de garantir la reproductibilité des résultats puis de faire appel avec ce générateur aux différentes lois qu'on souhaite utiliser. Dans le cas markoviens, on sera limités aux lois exponentielles.

In [3]:
?? np.random.default_rng

Signature:       np.random.default_rng(seed=None)
Call signature:  np.random.default_rng(*args, **kwargs)
Type:           cython_function_or_method
String form:    <cyfunction default_rng at 0x7fa68ccdda40>
Docstring:     
default_rng(seed=None)
Construct a new Generator with the default BitGenerator (PCG64).

    Parameters
    ----------
    seed : {None, int, array_like[ints], SeedSequence, BitGenerator, Generator}, optional
        A seed to initialize the `BitGenerator`. If None, then fresh,
        unpredictable entropy will be pulled from the OS. If an ``int`` or
        ``array_like[ints]`` is passed, then all values must be non-negative and will be
        passed to `SeedSequence` to derive the initial `BitGenerator` state. One may also
        pass in a `SeedSequence` instance.
        Additionally, when passed a `BitGenerator`, it will be wrapped by
        `Generator`. If passed a `Generator`, it will be returned unaltered.

    Returns
    -------
    Generator
        The

In [4]:
rng = np.random.default_rng(seed=42)

In [5]:
l = 2.

In [6]:
rng.exponential(1/l, 10)

array([1.2021043 , 1.16809483, 1.1923805 , 0.13989714, 0.0432187 ,
       0.72633026, 0.70498035, 1.56214798, 0.0396471 , 0.52328042])

Attention ici au fait que le premier paramètre de `rng.exponential` correspond à $1/\lambda$ et non à $\lambda$ dans le cours. Pour plus d'information vous êtes invités à consulter l'aide : `? rng.exponential`.

------------

## Modèle de travail

Afin de garantir une certaine généricité de fonctionnement, nous allons encoder le résultats d'une simulation dans un `dataframe` qui permette des post-traitements statiques sur les données d'une simulation. Cela permet également de stocker le résultat d'une simulation pour consultations ultérieures notamment si l'on souhaite faire des comparatifs entre différents choix de simulations ou de paramètrage. 

On va pour la suite charger la bibliothèque `pandas` qui permet de manipuler et stocker facilement des données tabulaires.

In [7]:
import pandas as pd

On qualifie **d'Agent** une notion qui correspond à celle de client
dans le folklore d'une file d'attente au guichet. Il est défini par
les attributs suivants

- `id` : son identifiant, un `int`.
- `t_arval_queue` : temps d'arrivée dans le système d'attente, `float`.
- `t_arval_srv` : temps d'accès à un serveur, `float`.
- `t_depart_sys` : temps de départ du système, `float`.

Dans l'implémentation qu'on propose chaque agent correspond aux
premières colonnes d'une ligne dans un dataframe, nommé `tops`. Les
colonnes concernées seront respectivement nommées
`t_arval_queue`, `t_arval_srv` et `t_depart_sys`. L'attribut `id` correspond à une indexation sur les lignes.

Les éléments décrits de `tops` correspondent aux tops d'un chronomètres en observation d'une file d'attente. On se laisse le droit, même si cela n'est pas très orthodoxe, de rajouter une information d'un autre ordre dans les colonnes de `tops`.

In [8]:
tops = pd.DataFrame(rng.uniform(.1, size=(10, 3)), 
                    columns = ['t_arval_queue', 't_arval_srv', 't_depart_sys'])

In [9]:
tops

,t_arval_queue,t_arval_srv,t_depart_sys
0,0.433718,0.934088,0.679479
1,0.840485,0.499073,0.304515
2,0.599126,0.157436,0.844868
3,0.668498,0.782279,0.419073
4,0.973628,0.903809,0.800545
5,0.275175,0.520049,0.139423
6,0.238861,0.714744,0.770286
7,0.970759,0.393243,0.433414
8,0.522600,0.270524,0.216929
9,0.528134,0.304218,0.702833


Le code précédent ne vise qu'à vous fixer la forme de `tops`. Il devrait être clair pour vous que le remplissage de `tops` dans le cadre de la simulation d'une file d'attente ne sera pas effectuer de cette façon. 

-------

# Simuler une file M/M/1

Il est bien entendu illusoire de souhaiter simuler une file M/M/1 qui s'exécute infiniment, il s'agit ici d'avoir une file pour laquelle aucun refus ne peut avoir lieu indépendamment des congestions qu'elle peut subir. Le nombre d'agents intervenants dans la simulation ne sont dans ce contexte qu'un échantillon de la population infinie considérée.

La classe qui suit, cherche à standardiser la façon avec laquelle on souhaite étudier / formaliser nos différentes simulation de files d'attentes

In [31]:
class mm1():
    """
    A class representing an mm1 queue. 

    Attributes :

        lamda (float)   : parameter of exponential law corresponding to interarrival times.
        mu (float)      : parameter of exponential law corresponding to service times.
        gen (np.random) : a random generator
        test_z (int)    : test size
        tops            : dataframe containing queue and service arrival times, and departure times for each agent.
    """
    
    
    def __init__(self, lamda, mu, gen, test_z=100) :
        """
        Initializes a news instance of the class mm1.
        
        Args :
            lamda  : parameter of exponential law corresponding to interarrival times.
            mu     : parameter of exponential law corresponding to service times.
            gen    : a random generator
            test_z : test size. 
        """
        
        #Initializing metadata
        self.lamda = lamda
        self.mu = mu
        self.gen = gen
        self.t_size = test_z
        
        #Initializin tops dataframe
        column_names = ['t_arival', 't_service', 't_depart']
        self.tops = pd.DataFrame(np.empty((self.t_size, 3),dtype=object), columns=column_names)
        
    def run(self):
        """
        Simulates the mm1 queuing system

        Modifies the attribute tops following a simulation of mm1 queue respecting interarrival and 
        outgoing intensities given by corresponding attributes. Besides filling in queue arrival times, 
        service arrival times and departure times for each agent, it adds columns to tops corresponding 
        to sojourn time (t_sojourn), waiting time (t_waiting) and service time (t_service) of each agent.
        """
        #FIXME

        self.tops["t_arival"][0] = self.gen.exponential(self.lamda)
        self.tops["t_service"][0] = self.tops["t_arival"][0]
        self.tops["t_depart"][0] = self.tops["t_service"][0] + self.gen.exponential(self.mu)
        for line in range(1, self.t_size):
            self.tops["t_arival"][line] = self.tops["t_arival"][line - 1] + self.gen.exponential(self.lamda)
            self.tops["t_service"][line] = max(self.tops["t_arival"][line],self.tops["t_depart"][line - 1])
            self.tops["t_depart"][line] = self.tops["t_service"][line] + self.gen.exponential(self.mu)
        
    
    
    def counts(self, t_intervals):
        """
        Computes number of agents in system at all given times.

        Args :
            t_intervals : size of time laps between two successive tops.

        Returns :
            A dataframe indexed by time laps of size t_intervals all along total queue simulation time, 
            its three columns contain number of agents in system (ag_in_sys), in arrival queue (ag_in_queue) and
            in service (ag_in_service).
        """                
        # FIXME

    def stats(self):
        """
        Computes statistics of a current simulation of queue.

        Returns:
            A dataframe indexed by standard statistics of interest mean sojourn time (mean_sojourn_time), 
            mean waiting time (mean_waiting_time) and mean service time (mean_service_time).
        """
        #FIXME

1. Implémenter les fonctions laissées dans le code.

2. Simuler le comportement d'une file mm1 avec les paramètres de votre choix. On note `mm1_counts` et `mm1_stats` les variables qui stockent les retours des fonctions `counts` et `stats` pour une exécution de votre choix de `mm1`. Vous pourrez représenter graphiquement les résultats de vos tests à l'aide du code ci-dessous

In [11]:
import matplotlib.pyplot as plt

In [32]:
a = mm1(3,1,rng)
a.run()
print(a.tops)

# fig, axes = plt.subplots(3, 1, figsize=(16, 12))
# counts = ['ag_in_sys','ag_in_queue','ag_in_service']
# labels = ['system', 'queue', 'service']
# colors = ['red', 'blue', 'black']

# for i in range(3):
#     mm1_counts.plot(y= counts[i], 
#                    use_index=True, ax=axes[i], linewidth=1.5,drawstyle='steps-mid', color=f'{colors[i]}',
#                    ylabel=f'Agent number in {labels[i]}', xlabel='Time', ylim=[0,max(mm1_counts['ag_in_sys']+1)])
    

      t_arival   t_service    t_depart
0     2.938168    2.938168     3.60806
1     7.264257    7.264257     8.00427
2    15.486065   15.486065   15.547111
3    31.562662   31.562662   32.390502
4     32.44718    32.44718   33.033742
..         ...         ...         ...
95  304.749193  304.749193  305.102143
96  306.432019  306.432019  307.082851
97  307.245168  307.245168  308.215376
98   310.77262   310.77262  312.330338
99  311.461632  312.330338  313.322339

[100 rows x 3 columns]


/tmp/ipykernel_25081/303537156.py:47: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.tops["t_arival"][0] = self.gen.exponential(self.lamda)
/tmp/ipykernel_25081/303537156.py:48: FutureWarning: ChainedAssignmentError: behaviour will chang

3. Comparer les résultats que vous obtenez à l'aide de votre simulateur aux résultats théorques attentdus.

-----------

# Le cas d'une file M/M/1/K

1. Quels sont les comportement qu'on souhaite retrouver dans le cas des files M/M/1/K qui sont exclus du cas M/M/1 précédents ? 

2. Adapter le code `mm1` précédent au cas d'une file d'attente `mm1k`. 

3. Comparer ce que vous obtenez aux résultats théoriques.

4. Étuder l'impact de différentes politiques de priorisation sur les statistiques de comportement d'une file d'attente M/M/1/K

-------

# Files d'attentes déterministes

1. Simuler une file d'attente déterministe de type D/D/1/K.

2. Étudier le taux de refus et le taux de saturation d'une file D/D/1/K en fonction des pas des interarrivées et de ceux de service. 

3. Que pouvez-vous en dire ?

--------